In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

In [1]:
from sklearn.base import ClusterMixin, BaseEstimator
from numpy.linalg import norm
from sklearn.metrics.pairwise import distance_metrics
import copy

class Kmeans(BaseEstimator, ClusterMixin): 
    
    def __init__(self, k=2, metric='euclidean', max_iter=1000, random_state=None):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter

    def fit(self, X, y=None):
        """
        Процедура обучения k-means
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        self.labels = np.empty(X.shape[0]).astype(int)
        new_labels = np.empty(X.shape[0]).astype(int)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        self.centroids = np.empty((self.k, X.shape[1]))
        new_centroids = np.empty((self.k, X.shape[1]))
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            self.centroids[:, i] = min + np.random.random(self.k)*(max-min)
        
        ## Your Code Here
                
        for it in xrange(self.max_iter):
            
            #заполняем new_labels - ближайший центроид для каждого объекта
            new_labels = distance_metrics()[self.metric](self.centroids, X).argmin(axis=0)
                        
            #считаем новые координаты центроидов
            for cen in xrange(self.k):
                new_centroids[cen] = np.average(X[self.labels == cen], axis=0)
            
            #если нет ни одной точки в кластере, снова случайно выставляем этот центроид
            empty_clusters = np.argwhere(np.isnan(new_centroids).any(axis=1))
            new_centroids[empty_clusters] = self.centroids[empty_clusters]         
            
            #если среднее смещение центроида меньше eps, то заканчиваем
            #delta = float(np.trace(distance_metrics()[self.metric](self.centroids, new_centroids))/self.k)
            #eps = 0.001
            #if delta < eps:
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            if np.array_equal(new_labels, self.labels):
                self.centroids = copy.deepcopy(new_centroids)
                self.labels = copy.deepcopy(new_labels)
                #print 'iterations = ', it
                break
            
            self.centroids = copy.deepcopy(new_centroids)
            self.labels = copy.deepcopy(new_labels)

        return self

    def predict(self, X, y=None):
        """
        Процедура предсказания кластера
        
        Возвращает метку ближайшего кластера для каждого объекта
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        return distance_metrics()[self.metric](self.centroids, X).argmin(axis=0)
